In [37]:
import nibabel as nib
import numpy as np

加入其他被试时只需要添加sub_flag即可

In [56]:
sub_flag = ['sub-03','sub-06','sub-07','sub-08','sub-09',
            'sub-11','sub-12','sub-13','sub-14','sub-15','sub-16','sub-17','sub-18','sub-19','sub-20',
            'sub-21','sub-22','sub-23','sub-24','sub-25','sub-26','sub-27','sub-28','sub-29','sub-30']
fstat_flag = ['fstat1.dtseries.nii','fstat2.dtseries.nii','fstat3.dtseries.nii']

In [41]:
container_f1 = np.zeros((1,91282))
container_f2 = np.zeros((1,91282))
container_f3 = np.zeros((1,91282))
for sub in sub_flag:
    file_path = '/nfs/t2/raven/data/bold/derivatives/ciftify/{}/MNINonLinear/Results/ses-raven_task-action/f_test_file_summary/flameoResultdir.feat/'.format(sub)
    for fstat in fstat_flag:
        dof_path = file_path + fstat
        if fstat == 'fstat1.dtseries.nii':
            container_f1 = container_f1 + nib.load(dof_path).get_fdata()
        if fstat == 'fstat2.dtseries.nii':
            container_f2 = container_f2 + nib.load(dof_path).get_fdata()
        if fstat == 'fstat3.dtseries.nii':
            container_f3 = container_f3 + nib.load(dof_path).get_fdata()
avg_f1 =  np.divide(container_f1,len(sub_flag))   
avg_f2 =  np.divide(container_f2,len(sub_flag))   
avg_f3 =  np.divide(container_f3,len(sub_flag))   

In [42]:
output = '../result/group_avg/' + 'attribute_group_avg.dtseries.nii'
img = nib.Cifti2Image(avg_f1, nib.load(dof_path).header)
nib.save(img, output)

In [43]:
output = '../result/group_avg/' + 'rules_group_avg.dtseries.nii'
img = nib.Cifti2Image(avg_f2, nib.load(dof_path).header)
nib.save(img, output)

In [44]:
output = '../result/group_avg/' + 'interaction_group_avg.dtseries.nii'
img = nib.Cifti2Image(avg_f3, nib.load(dof_path).header)
nib.save(img, output)

In [46]:
from scipy import stats
import numpy as np

# 假设你的组平均效应数据存储在mean_effect数组中

# 假设零假设为组平均效应为零
null_hypothesis = 0

# 进行单样本t检验
t_statistic, p_value = stats.ttest_1samp(avg_f1, null_hypothesis)

/nfs/h1/userhome/liyifan/.local/lib/python3.8/site-packages/scipy/stats/_axis_nan_policy.py:527: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, axis=axis, **kwds)
/nfs/h1/userhome/liyifan/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/nfs/h1/userhome/liyifan/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: invalid value encountered in multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


In [50]:
avg_f1.shape

(1, 91282)

In [65]:
import os
import numpy as np
from scipy import stats
import nibabel as nib

# 被试目录路径
subjects_dir = '/nfs/t2/raven/data/bold/derivatives/ciftify'

# 初始化结果数组
p_values = []

# 存储每个组的数据
group_data = []

# 遍历每个被试目录
for subject in sub_flag:
    subject_dir = os.path.join(subjects_dir, subject, 'MNINonLinear', 'Results', 'ses-raven_task-action','f_test_file_summary')
    file_name1 = '{}_ses-raven_task-action_level2_cope_A2R1-A1R1-A2R2+A1R2_hp200_s4.dscalar.nii'.format(subject)
    file_name2 = '{}_ses-raven_task-action_level2_cope_A2R3-A1R3-A2R2+A1R2_hp200_s4.dscalar.nii'.format(subject)
    file_name3 = '{}_ses-raven_task-action_level2_cope_A3R1-A1R1-A3R2+A1R2_hp200_s4.dscalar.nii'.format(subject)
    file_name4 = '{}_ses-raven_task-action_level2_cope_A3R3-A1R3-A3R2+A1R2_hp200_s4.dscalar.nii'.format(subject)
    
    # 要比较的多个文件
    file_names = []
    file_names.append(file_name1)
    file_names.append(file_name2)
    file_names.append(file_name3)
    file_names.append(file_name4)

    # 存储当前被试的数据
    subject_data = []

    # 加载图像数据
    for file_name in file_names:
        file_path = os.path.join(subject_dir, file_name)
        img = nib.load(file_path)
        data = img.get_fdata()
        subject_data.append(data)

    # 将当前被试的数据添加到组数据列表中
    group_data.append(subject_data)

# 执行方差分析（ANOVA）
f_statistic, p_value = stats.f_oneway(*group_data)
# 打印p值

In [64]:
output = '../result/group_avg/' + 'interaction_p_map.dtseries.nii'
img = nib.Cifti2Image(p_value, nib.load(dof_path).header)
nib.save(img, output)